***Импорт библиотек***

In [255]:
import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="darkgrid")

from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, RobustScaler
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import balanced_accuracy_score, cohen_kappa_score

from imblearn.over_sampling import SMOTE
from collections import Counter

import warnings
warnings.simplefilter('ignore')

# np.set_printoptions(suppress=True)

# settings to display all columns
pd.set_option("display.max_columns", None)

# фиксация RANDOM_SEED для воспроизведения эксперимента
RANDOM_SEED = 42

# фиксация версии пакетов

import os
for dirname, _, filenames in os.walk('/kaggle/input\sf-dst-scoring'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
    



***Загрузка и предварительный осмотр данных***

In [256]:
DATA_DIR = '/kaggle/input/sf-dst-scoring/'
train = pd.read_csv(DATA_DIR+'/train.csv')
test = pd.read_csv(DATA_DIR+'test.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [257]:
# тренировочные данные
print(train.info())
print(f'Размер тренировочного датасета: {train.shape}')
train.head(3)

In [258]:
# тестовые данные
print(test.info())
print(f'Размер тестового датасета: {test.shape}')
test.head(3)

In [259]:
# данные представления
print(sample_submission.info())
print(f'Size dataset: {sample_submission.shape}')
sample_submission.head(3)

In [260]:
# объединяем данные в один датасет
train['sample'] = 1 # тренировочный
test['sample'] = 0 # тестовый
test['default'] = -1

data = pd.concat([train, test], ignore_index=True)
data.tail(5)

In [261]:
# данные общего датасета
print(data.info())
print(f'Размер объединенного датасета: {data.shape}')
data.head(3)

***Рассмотрим данные датасета***
* client_id - идентификатор клиента
* education - уровень образования
* sex - пол заемщика
* age - возраст заемщика
* car - флаг наличия автомобиля
* car_type - флаг автомобиля иномарки
* decline_app_cnt - количество отказанных прошлых заявок
* good_work - флаг наличия "хорошей" работы
* bki_request_cnt - количество запросов клиента в БКИ о своей кредитной истории
* home_address - категоризатор домашнего адреса
* work_address - категоризатор рабочего адреса
* income - доход заемщика
* foreign_passport - наличие загранпаспорта
* sna - звязь заемщика с клиентами банка
* first_time - давность наличия информации о заемщике
* score_bki - скоринговый балл по данным из БКИ
* region_rating - рейтинг региона
* app_date - дата подачи заявки
* default - флаг дефолта по кредиту

***Предобработка***

In [262]:
def missing_value(df, missing_percent):
    if df.isnull().sum().sum() > 0:
        mask_total = df.isnull().sum().sort_values(ascending=False)
        total = mask_total[mask_total > 0]
        
        mask_percent = df.isnull().mean().sort_values(ascending=False)
        percent = mask_percent[mask_percent > 0]
        
        series = mask_percent[mask_percent > missing_percent]
        columns = series.index.to_list()
        
        missing_data = pd.DataFrame(pd.concat([total, round(percent*100, 2)], axis=1, keys=['Количество', '%']))
        print('Сумма и процент значений NaN:')
        display(missing_data)
    else:
        print('NaN значения не найдены.')
        
        
def an_katcol(df, col):
    print(f'Признак: \033[1m\033[30m{col}')
    
    fig, axes = plt.subplots(ncols=2)
    sns.countplot(df[col], hue='default', data = df, ax=axes[0])
    axes[0].set_title('График распределения признака')
    sns.barplot(x=col, y='default', data=df, ax=axes[1])
    axes[1].set_title('Влияние признака на дефолт')
    fig.tight_layout(rect=[-1, 0.03, 1, 0.95])
    plt.show()
    
    
    
def an_katcol_box(df, col1, col2):
    sns.boxplot(x=col1, y=col2, data=df, showfliers=False)
    plt.show();
        

***Изучим данные***

In [263]:
# Типы данных
pd.DataFrame(data.dtypes, columns=['Тип переменной']).transpose()

In [264]:
# Проверим пропуски
missing_value(data,0)

Пропуски встречаются в информации об образовании заемщика, их необходимо заполнить.

целевая переменная default, обратим внимание на ее распределение. 

In [265]:
#  Проверка несбалансированных данных
display(train['default'].value_counts())
count_no_def = len(train[train['default'] == 0])
count_def = len(train[train['default'] == 1])
print(f'Процент не выплативших кредит {round((count_def/(count_def + count_no_def))*100, 2)} %')
print(f'Процент выплативших кредит {round((count_no_def/(count_def + count_no_def))*100, 2)} %')
print(f'Соотношение положительных и отрицательных примеров {round((count_def/count_no_def)*100, 2)} %')
sns.countplot(train['default'], palette = 'magma')

***Дополнительные исследования***

In [266]:
round(train.drop(['client_id', 'sample'], 1).groupby('default').mean(), 3)

***Наблюдения***
* у выплативших долг средний возраст выше
* у невыплативших больше отказов
* у выплативших работа лучше
* высокий бал по данным из БКИ подозрителен
* у выплативших выше рейтинг района а с показательями адреса дома и работы все иначе
* доход у выплативших выше и их информация дольше хранится дольше, а вот связь с другими клиентами банка не самый лучший показатель

***Анализ, очистка и подготовка данных***


In [267]:
print(f'Количество уникальных значений в переменных:')
unic_val, val, idx = [], [], []
for column_name in data.columns:
    unic_val.append(data[column_name].nunique())
    val.append(data[column_name].unique())
    idx.append(column_name)
df = pd.concat([pd.Series(idx), pd.Series(unic_val), pd.Series(val)], axis=1, keys=['Перменная', 'Количество уникальных значений', 'Уникальные значения'])
display(df)

In [268]:
round(train.drop(['client_id', 'sample'], 1).groupby('education').mean(), 2)

In [269]:
round(train.drop(['client_id', 'sample'], 1).groupby('foreign_passport').mean(), 2)

In [270]:
round(train.drop(['client_id', 'sample'], 1).groupby('sex').mean(), 2)

***Обработка пропусков и замена на чаще встречающиеся значения***

In [271]:
data['education'].fillna(data['education'].mode()[0], inplace=True)
display(pd.DataFrame(data['education'].value_counts()))

missing_value(data, 0)

In [272]:
data['app_date'] = pd.to_datetime(data['app_date'])

display(data['app_date'].max())
display(data['app_date'].min())

In [273]:
min_date = data['app_date'].min()
data['days'] = data['app_date'].apply(lambda x: (x - min_date).days)
data['days_relative'] = data['days']/120
data.drop(['app_date', 'days'], 1, inplace=True)

***Работа с переменными***

In [274]:
# бинарные переменные
bin_cols = ['sex','car','car_type','good_work','foreign_passport']
# категориальные переменные
cat_cols = ['education','home_address','work_address','sna','first_time','region_rating']
# числовые переменные
num_cols = ['age','decline_app_cnt','bki_request_cnt','income','score_bki','days_relative']

In [275]:
fig, axes = plt.subplots(2, 3, figsize=(25,15))

for i,col in enumerate(num_cols):
    sns.distplot(data[col], kde=False, ax=axes.flat[i], color='k')

In [276]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
for i, col in enumerate(['decline_app_cnt', 'bki_request_cnt', 'income']):
    data[col+'_log'] = data[col].apply(lambda w: np.log(w + 1))
    sns.distplot(data[[col+'_log']][data[[col+'_log']] > 0], kde=False, ax=axes.flat[i], color='k')

In [277]:
fig, axes = plt.subplots(1, 6, figsize=(15, 6))
plt.subplots_adjust(wspace=0.5)
for i in range(len(num_cols)):
    sns.boxplot(x='default', y=num_cols[i], data=data[data['sample'] == 1], orient='v', ax=axes[i], showfliers=False)

***Выводы по числовым переменным***

Дефолтные клиенты в среднем младше, имеют большее количество отказов, большее количество запросов в БКИ, доход ниже, скоринговый балл из БКИ выше, а заявки поданы по дате раньше.

***Бинарные переменные***

In [278]:
hue = data['default']
data_fig = data[data['sample'] == 1]
fig, axes = plt.subplots(2, 3, figsize=(25, 15))
for i, col in enumerate(bin_cols):
    sns.countplot(x=data[col], hue=hue, data=data_fig, ax=axes.flat[i], palette='magma')

In [279]:
pd.DataFrame(round(data[['sex', 'default']].value_counts(normalize=True)*100, 2), columns=['%']).transpose()

In [280]:
pd.concat([pd.DataFrame(round(data[data['sex'] == 'F']['default'].value_counts(\
                                                                              normalize=True)*100, 2)),
          pd.DataFrame(round(data[data['sex'] == 'M']['default'].value_counts(\
                                                                            normalize=True)*100, 2))],\
         axis=1, keys=['F', 'M'])

In [281]:
pd.DataFrame(round(data[['car', 'default']].value_counts(normalize=True)*100, 2), columns=['%']).transpose()

In [282]:
pd.DataFrame(round(data[['car_type', 'default']].value_counts(normalize=True)*100, 2), columns=['%']).transpose()

In [283]:
pd.DataFrame(round(data[['foreign_passport', 'default']].value_counts(normalize=True)*100, 2), columns=['%']).transpose()

***Выводы***

Несмотря на то, что женщины берут кредиты чаще, количество дефолтов у них не выше. Клиенты без машин берут больше кредитов, а возвращают их реже. Клиенты на иномарках - наоборот. Тоже самое с работой и наличием заграна.

In [284]:
# кодируем признаки

label_encoder = LabelEncoder()
label_bin = {}
for column in bin_cols:
    data[column] = label_encoder.fit_transform(data[column])
    label_bin[column] = dict(enumerate(label_encoder.classes_))
print(label_bin)

data[bin_cols].head()

***Категориальные переменные***

In [285]:
for i in cat_cols:
    an_katcol(data[data['default'] != -1], i)

In [286]:
k = ['income', 'age', 'region_rating', 'score_bki']
for i in range(len(k)):
    an_katcol_box(data, 'education', k[i])

In [287]:
k = ['income', 'age', 'score_bki']
for i in range(len(k)):
    an_katcol_box(data, 'region_rating', k[i])

***Выводы***

Клиенты с хорошим образование реже берут кредиты и лучше их платят, у них выше доход и лучше районы.

In [288]:
print(list(cat_cols))
for col in cat_cols:
    print(data[col].unique())

In [289]:
label_cat = {}
for column in cat_cols:
    data[column] = label_encoder.fit_transform(data[column])
    label_cat[column] = dict(enumerate(label_encoder.classes_))
    print(label_cat)
    
data[cat_cols].head()

***Корреляционный анализ***

In [290]:
fig, ax = plt.subplots(figsize=(12, 7))
sns.heatmap(data[num_cols].corr(), vmin=0, vmax=1, annot=True, fmt='.3f', cmap='Greens')

Можно оставить так.

In [291]:
fig, ax = plt.subplots(figsize=(12, 7))
sns.heatmap(data[bin_cols].corr(), vmin=0, vmax=1, annot=True, fmt='.3f', cmap='Greens')

Признаки car и car_type скоррелированы

In [292]:
fig, ax = plt.subplots(figsize=(12, 7))
sns.heatmap(data[cat_cols].corr().abs(), vmin=0, vmax=1, annot=True, fmt='.3f', cmap='Greens')

Также скоррелированны признаки work_address и home_address, sna и first_time

***Создание признаков***

In [293]:
# Объдиним признаки car и car_type и переведем новый признак в категориальный.
# 0 - нет авто, 1 - есть отечественный, 2 - иномарка

data['car_comb'] = data['car'] + data['car_type']
data['car_comb'] = data['car_comb'].astype('category')

data.drop(['car', 'car_type'], 1, inplace=True)
bin_cols.remove('car_type')
bin_cols.remove('car')
cat_cols.append('car_comb')

print(data['car_comb'].unique())

data['car_comb'] = label_encoder.fit_transform(data['car_comb'])
data[cat_cols].head()

In [294]:
# Новый признак о среднем доходе по возрасту
mean_inc_age = data.groupby('age')['income'].mean().to_dict()
data['mean_income_age'] = data['age'].map(mean_inc_age)

# Максимальный доход для конкретного возраста
max_income = data.groupby('age')['income'].max().to_dict()
data['max_income_age'] = data['age'].map(max_income)

# Нормализуем доход
data['normal_income'] = abs(
    (data['income'] - data['mean_income_age'])/data['max_income_age'])

data.drop(['mean_income_age', 'max_income_age'], axis=1, inplace=True)

# Добавим признак о среднем количестве отказанных заявок по возрастам
mean_dec_age = data.groupby('age')['decline_app_cnt'].mean().to_dict()
data['mean_decline_age'] = data['age'].map(mean_dec_age)

# Максимальное количество отказов
max_decline = data.groupby('age')['decline_app_cnt'].max().to_dict()
data['max_decline_age'] = data['age'].map(max_decline)

# Нормализуем количество отказов
data['normal_decline'] = abs(
    (data['decline_app_cnt'] - data['mean_decline_age'])/data['max_decline_age'])
data['normal_decline'].fillna(data['normal_decline'].median(), inplace=True)

# Добавим признак о среднем количестве запросов в БКИ
mean_bki_age = data.groupby('age')['bki_request_cnt'].mean().to_dict()
data['mean_bki_age'] = data['age'].map(mean_bki_age)

# Максимальное количество запросов в БКИ для конкретного возраста
max_bki = data.groupby('age')['bki_request_cnt'].max().to_dict()
data['max_bki_age'] = data['age'].map(max_bki)

# Нормализуем количество запросов
data['normal_bki'] = abs(
    (data['bki_request_cnt'] - data['mean_bki_age'])/data['max_bki_age'])
data['normal_bki'].fillna(data['normal_bki'].median(), inplace=True)

data.drop(['mean_decline_age', 'max_decline_age','mean_decline_age', 'max_decline_age', 'mean_bki_age', 'max_bki_age'], axis=1, inplace=True)

In [295]:
num_cols_new = ['normal_income', 'normal_decline', 'normal_bki']
for i in num_cols_new:
    num_cols.append(i)
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i, col in enumerate(num_cols_new):
    sns.distplot(data[col], ax=axes.flat[i], color='k')

In [296]:
max_age = data['age'].max()
data['age'] = data['age']/max_age
sns.distplot(data['age'], color='k')

In [297]:
data.drop(['income', 'decline_app_cnt', 'bki_request_cnt'], 1, inplace=True)

In [298]:
num_cols.remove('income')
num_cols.append('income_log')
num_cols.remove('decline_app_cnt')
num_cols.append('decline_app_cnt_log')
num_cols.remove('bki_request_cnt')
num_cols.append('bki_request_cnt_log')

In [299]:
fig, ax = plt.subplots(figsize=(18, 7))
sns.heatmap(data.drop(['client_id', 'sample'], 1).corr().abs(), vmin=0, vmax=1, annot=True, fmt='.3f', cmap='Greens')

In [300]:
data.drop(['normal_decline'], 1, inplace=True)
num_cols.remove('normal_decline')

***Значимость числовых переменных***

В основе метода лежит однофакторный дисперсионный анализ. Основа процедуры - обобщение результатов двух выборочных t-тестов для независимых выборок.

В качестве меры значимости будем использовать значение f-статистики. Чем оно выше, тем меньше вероятность того, что средние значения не отличаются, и тем важнее данный признак для линейной модели.

In [301]:
df = data[data['sample'] == 1]
imp_num = pd.Series(f_classif(df[num_cols], df['default'])[0], index=num_cols)
imp_num.sort_values(inplace=True)
imp_num.plot(kind='barh')

In [302]:
f_classif(df[num_cols], df['default'])

Из числовых признаков самый значимый score_bki, а самый незначимый normal_income

***Значимость категориальных признаков***

In [303]:
df = data[data['sample'] == 1]
imp_cat = pd.Series(mutual_info_classif(df[bin_cols + cat_cols], df['default'], discrete_features=True), index=bin_cols + cat_cols)
imp_cat.sort_values(inplace=True)
imp_cat.plot(kind='barh')

In [304]:
f_classif(df[bin_cols + cat_cols], df['default'])

Самый значимый признак - sna, самый незначимый - sex

In [305]:
data = pd.get_dummies(data, prefix=cat_cols, columns=cat_cols)

In [306]:
data.info()

In [307]:
data.columns[14:43]

In [308]:
cat_cols_new = list(data.columns[14:43])
len(data.columns[14:43])

***Обновим списки переменных***

In [309]:
# Уберем из числовых переменных переведенные в категориальные переменные
print(f'числовые переменные: {num_cols}\n')
# добавим к бинарным переменным признак пропусков в данных
print(f'бинарные переменные: {bin_cols}\n')
# добавим к категориальным переменным те, что перевели из числовых
print(f'категориальные переменные: {cat_cols_new}')

In [310]:
data

***Построение модели***

Подготовка данных и обучение модели

In [311]:
# Делим датасет на тест и трейн

train_df = data.query('sample == 1').drop(['sample', 'client_id'], axis=1)
test_df = data.query('sample == 0').drop(['sample', 'client_id'], axis=1)
train_df.shape, test_df.shape

***Выбросы и стандартизация числовых переменных***

In [312]:
for col in num_cols:
    perc25 = data[col].quantile(0.25)
    perc75 = data[col].quantile(0.75)
    IQR = perc75 - perc25
    
    print('Column:', col)
    print(
        f'25%: {perc25}, 75%: {perc75}, IQR: {IQR}, \
        Borderline: [{perc25 - 1.5*IQR}, {perc75 + 1.5*IQR}]')
    print(
        f'min: {data[col].min()}, median: {data[col].median()}, \
        max: {data[col].max()}')
    print()

Выбросы незначительны, осталось стандартизировать

In [313]:
scaler = StandardScaler()
X_num = scaler.fit_transform(train_df[num_cols].values)
X_num_test = scaler.transform(test_df[num_cols].values)

In [314]:
# Объединим числовые, бинарные и категориальные переменные в одно признаковое пространство
# разделим при этом признаки и целевую переменную

X = np.hstack([X_num, train_df[bin_cols].values, train_df[cat_cols_new].values])
Y = train_df['default'].values

X_test_data = np.hstack([X_num_test, test_df[bin_cols].values, test_df[cat_cols_new].values])
Y_test_data = test_df['default'].values

In [315]:
#  разделим данные для обучения
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=RANDOM_SEED)
print(X.shape, X_train.shape, X_test.shape, Y.shape, y_train.shape, y_test.shape)

***Моделирование и оценка качества модели***

Обучим модель на стандартных настройках логистической регрессии (базовая модель)

In [316]:
model_0 = LogisticRegression(class_weight='balanced', max_iter=2000, random_state=RANDOM_SEED)
model_0.fit(X_train, y_train)
y_pred_0 = model_0.predict(X_test)

In [317]:
probs_0 = model_0.predict_proba(X_test)
probs_0 = probs_0[:, 1]

fpr, tpr, threshold = roc_curve(y_test, probs_0)
roc_auc = roc_auc_score(y_test, probs_0)
conf_mat_0 = confusion_matrix(y_test, y_pred_0)

plt.figure(figsize=(8, 5))
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label='Regression')
plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc='lower right')
plt.show()

metrics_0 = {'accuracy': round(accuracy_score(y_test, y_pred_0), 4), 
             'balanced_accuracy_score': round(balanced_accuracy_score(y_test, y_pred_0), 4), 
             'precision_score': round(precision_score(y_test, y_pred_0), 4), 
             'Sensitivity_reccall_score': round(recall_score(y_test, y_pred_0), 4), 
             'Specificity_1-fpr': round(1 - fpr.mean(), 4), 
             'f1': round(f1_score(y_test, y_pred_0), 4), 
             'fpr': round(fpr.mean(), 4), 
             'tpr': round(tpr.mean(), 4), 
             'roc_auc': round(roc_auc, 4), 
             'cohen_kappa_score': round(cohen_kappa_score(y_test, y_pred_0), 4)}

display(pd.DataFrame(metrics_0.items(), columns=['Показатель', 'Значение'], index=range(1, 11)))
print(f'Доля правильных ответов классификатора на обучающей выборке: {round(model_0.score(X_train, y_train), 3)}')
class_names = ['NoDefault-0', 'Default-1']
df_cm = pd.DataFrame(conf_mat_0, index=class_names, columns=class_names)
print(f'Матрица ошибок')
display(df_cm)
plt.figure(figsize=(8, 5))
group_names = ['True Neg', 'False Pos', 'False Neg', 'True Pos']
group_counts = ['{0:0.0f}'.format(value) for value in confusion_matrix(y_test, y_pred_0).flatten()]
labels = [f'{v1}\n{v2}' for v1, v2 in zip(group_names, group_counts)]
labels = np.asarray(labels).reshape(2, 2)
sns.heatmap(df_cm, annot=labels, fmt='', cmap='BuGn_r', linewidths=0.01)
plt.ylabel('True label')
plt.xlabel('Predicted label')

***Валидация на hold-out выборке***

In [318]:
X_tr, X_valid, y_tr, y_valid = train_test_split(X_train, y_train, test_size=0.3, shuffle=True, random_state=RANDOM_SEED)
print(X_train.shape, X_tr.shape, X_valid.shape, y_train.shape, y_tr.shape, y_valid.shape)

In [319]:
logres = LogisticRegression(class_weight='balanced', max_iter=2000, random_state=RANDOM_SEED).fit(X_tr, y_tr)
logres.fit(X_tr, y_tr)

y_pred_tr = logres.predict(X_tr)
f1_train = f1_score(y_tr, y_pred_tr)
print(f'[train] F1-score = {f1_train:.2f}')

y_pred = logres.predict(X_valid)
f1_val = f1_score(y_valid, y_pred)
print(f'[valid] F1-score = {f1_val:.2f}')

***Кросс-валидация***

In [321]:
logres = LogisticRegression(class_weight='balanced', max_iter=2000, random_state=RANDOM_SEED)
cv_metrics = cross_validate(logres, X_train, y_train, cv=5, scoring='f1_micro', return_train_score=True)
print(cv_metrics)

def plot_cv_metrics(cv_metrics):
    avg_f1_train, std_f1_train = cv_metrics['train_score'].mean(), cv_metrics['train_score'].std()
    avg_f1_valid, std_f1_valid = cv_metrics['test_score'].mean(), cv_metrics['test_score'].std()
    print('[train] F1-score = {:.2f} +/- {:.2f}'.format(avg_f1_train, std_f1_train))
    print('[valid] F1-score = {:.2f} +/- {:.2f}'.format(avg_f1_valid, std_f1_valid))
    
    plt.figure(figsize=(15, 5))
    
    plt.plot(cv_metrics['train_score'], label='train', marker='.')
    plt.plot(cv_metrics['test_score'], label='valid', marker='.')
    
    plt.ylim([0., 1.])
    plt.xlabel('CV iteration', fontsize=15)
    plt.ylabel('F1-score', fontsize=15)
    plt.legend(fontsize=15)
    
plot_cv_metrics(cv_metrics)

Показатели модели приемлемыеб переобучение не обнаружено

***Масштабирование***

In [322]:
os = SMOTE(random_state=RANDOM_SEED)
os_data_X, os_data_y = os.fit_resample(X_train, y_train)
print(sorted(Counter(os_data_y).items()))
os_data_y1 = pd.DataFrame(data=os_data_y, columns=['default'])

# Проверяем числа данных
print('Длина данных с передискретизацией', len(os_data_X))
print('Количество Недефолтов в данных с избыточной выборкой', len(os_data_y1[os_data_y1['default'] == 0]))
print('Количество дефолтов в данных с избыточной выборкой', len(os_data_y1[os_data_y1['default'] == 1]))
print('Доля Недефолтов в данных с избыточной выборкой составляет', len(os_data_y1[os_data_y1['default'] == 0])/len(os_data_X))
print('Доля дефолтов в данных с избыточной выборкой составляет', len(os_data_y1[os_data_y1['default'] == 1])/len(os_data_X))

In [323]:
# обучим модель
model_1 = LogisticRegression(max_iter=2000, random_state=RANDOM_SEED).fit(os_data_X, os_data_y)
y_pred_1 = model_1.predict(X_test)
probs_1 = model_1.predict_proba(X_test)
probs_1 = probs_1[:, 1]

fpr, tpr, thpeshold = roc_curve(y_test, probs_1)
roc_auc = roc_auc_score(y_test, probs_1)
conf_mat_1 = confusion_matrix(y_test, y_pred_1)

plt.figure(figsize=(8, 5))
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label='Regression')
plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc='lower right')
plt.show()

metrics_1 = {'accuracy': round(accuracy_score(y_test, y_pred_1), 4), 
             'balanced_accuracy_score': round(balanced_accuracy_score(y_test, y_pred_1), 4), 
             'precision_score': round(precision_score(y_test, y_pred_1), 4), 
             'Sencitivity_recall_score': round(recall_score(y_test, y_pred_1), 4),
             'Specifiti_1_fpr': round(1 - fpr.mean(), 4), 
             'f1': round(f1_score(y_test, y_pred_1), 4), 
             'fpr': round(fpr.mean(), 4), 
             'tpr': round(tpr.mean(), 4), 
             'roc_auc': round(roc_auc, 4), 
             'cohen_kappa_score': round(cohen_kappa_score(y_test, y_pred_1), 4)}
display(pd.DataFrame(metrics_1.items(), columns=['Показатель', 'Значение'], index=range(1, 11)))

print(f'Доля правильных ответов классификатора на обучающей выборке: {round(model_1.score(X_train, y_train), 3)}')

class_names = ['NoDefault-0', 'Default-1']
df_cm = pd.DataFrame(conf_mat_1, index=class_names, columns=class_names)
print(f'Матрица ошибок')

display(df_cm)
plt.figure(figsize=(8, 5))
group_names = ['True Neg', 'False Pos', 'False Neg', 'True Pos']
group_counts = ['{0:0.0f}'.format(value) for value in confusion_matrix(y_test, y_pred_1).flatten()]
labels = [f'{v1}\n{v1}' for v1, v2 in zip(group_names, group_counts)]
labels = np.asarray(labels).reshape(2, 2)
sns.heatmap(df_cm, annot=labels, fmt='', cmap='BuGn_r', linewidths=0.01)
plt.ylabel('True label')
plt.xlabel('Predicted label')

***Валидация на hold-out выборке***

In [324]:
X_tr, X_valid, y_tr, y_valid = train_test_split(os_data_X, os_data_y, test_size=0.3, shuffle=True, random_state=RANDOM_SEED)
print(X_train.shape, X_tr.shape, X_valid.shape, y_train.shape, y_tr.shape, y_valid.shape)

In [325]:
logres = LogisticRegression(max_iter=2000, random_state=RANDOM_SEED).fit(X_tr, y_tr)
logres.fit(X_tr, y_tr)

y_pred_tr = logres.predict(X_tr)
f1_train = f1_score(y_tr, y_pred_tr)
print(f'[train] F1-score = {f1_train:.2f}')

y_pred = logres.predict(X_valid)
f1_val = f1_score(y_valid, y_pred)
print(f'[valid] F1-score = {f1_val:.2f}')

***Кросс-валидация***

In [326]:
logres = LogisticRegression(max_iter=2000, random_state=RANDOM_SEED)
cv_metrics = cross_validate(logres, os_data_X, os_data_y, cv=5, scoring='f1_micro', return_train_score=True)
print(cv_metrics)

def plot_cv_metrics(cv_metrics):
    avg_f1_train, std_f1_train = cv_metrics['train_score'].mean(), cv_metrics['train_score'].std()
    avg_f1_valid, std_f1_valid = cv_metrics['test_score'].mean(), cv_metrics['test_score'].std()
    print('[train] F1-score = {:.2f} +/- {:.2f}'.format(avg_f1_train, std_f1_train))
    print('[valid] F1-score = {:.2f} +/- {:.2f}'.format(avg_f1_valid, std_f1_valid))
    
    plt.figure(figsize=(15, 5))
    
    plt.plot(cv_metrics['train_score'], label='train', marker='.')
    plt.plot(cv_metrics['test_score'], label='valid', marker='.')
    
    plt.ylim([0., 1.])
    plt.xlabel('CV iteration', fontsize=15)
    plt.ylabel('F1-score', fontsize=15)
    plt.legend(fontsize=15)
    
plot_cv_metrics(cv_metrics)

In [333]:
metrics = {'metrics_0': metrics_0, 'metrics_1': metrics_1}
pd.DataFrame(metrics)

In [334]:
conf_mat = {'conf_mat_0': [conf_mat_0], 'conf_mat_1': [conf_mat_1]}
pd.DataFrame(conf_mat)

***Submission***

In [335]:
sample_submission.tail(3)

In [336]:
X.shape

In [337]:
X_test_data.shape

In [338]:
y_probs = model_0.predict_proba(X_test_data)[:, 1]
test['default'] = y_probs
submission = test[['client_id', 'default']]
display(submission.sample(10))
display(submission.shape)

submission.to_csv('submission.csv', index=False)

Выбрана базовая модель, так как она показала лучший результат.